In [26]:
import qlib
import pandas as pd
from qlib.constant import REG_CN
from qlib.utils import exists_qlib_data, init_instance_by_config
from qlib.workflow import R
from qlib.workflow.record_temp import SignalRecord, PortAnaRecord
from qlib.utils import flatten_dict
from qlib.data.filter import NameDFilter
import wk_db
from qlib.data import D

import warnings
warnings.filterwarnings('ignore')

# %%
provider_uri = "/data/linq/.qlib/qlib_data/cn_data"  # target_dir
URI = '/home/linq/finance/qniverse/mlrun'
qlib.init(provider_uri=provider_uri, region=REG_CN)

[1364163:MainThread](2025-01-29 12:22:26,522) INFO - qlib.Initialization - [config.py:416] - default_conf: client.
[1364163:MainThread](2025-01-29 12:22:26,530) INFO - qlib.Initialization - [__init__.py:74] - qlib successfully initialized based on client settings.


[1364163:MainThread](2025-01-29 12:22:26,531) INFO - qlib.Initialization - [__init__.py:76] - data_path={'__DEFAULT_FREQ': PosixPath('/data/linq/.qlib/qlib_data/cn_data')}


In [27]:

EXP_NAME = 'MENSEMBLE'

In [28]:

with R.start(experiment_name=EXP_NAME, uri=URI):
    # recorder = R.get_recorder(experiment_name=EXP_NAME)
    recorder = R.get_recorder(recorder_id='2d4f94aa57704ca19c06632232cee757')
    pred_df = recorder.load_object("pred.pkl")
    report_normal_df = recorder.load_object("portfolio_analysis/report_normal_1day.pkl")
    positions = recorder.load_object("portfolio_analysis/positions_normal_1day.pkl")
    analysis_df = recorder.load_object("portfolio_analysis/port_analysis_1day.pkl")


[1364163:MainThread](2025-01-29 12:22:26,564) INFO - qlib.workflow - [exp.py:258] - Experiment 5 starts running ...


[1364163:MainThread](2025-01-29 12:22:26,591) INFO - qlib.workflow - [recorder.py:341] - Recorder 163f5cc34f68475ca78a1f44775b9842 starts running under Experiment 5 ...
[1364163:MainThread](2025-01-29 12:22:26,721) INFO - qlib.timer - [log.py:127] - Time cost: 0.000s | waiting `async_log` Done


In [29]:
positions

{Timestamp('2024-10-15 00:00:00'): {'_settle_type': 'None', 'position': {'cash': 200000, 'now_account_value': 200000.0}, 'init_cash': 200000},
 Timestamp('2024-10-16 00:00:00'): {'_settle_type': 'None', 'position': {'cash': 21253.673401631335, 'now_account_value': 199910.67150095035, 'SZ000552': {'amount': 22302.69770599803, 'price': 0.8471620678901672, 'weight': 0.094512210710336, 'count_day': 1}, 'SH601138': {'amount': 13654.67374269223, 'price': 1.3785755634307861, 'weight': 0.09416205451646445, 'count_day': 1}, 'SZ002993': {'amount': 17059.125151090586, 'price': 0.95772784948349, 'weight': 0.08172649875244921, 'count_day': 1}, 'SH601872': {'amount': 9014.07284636492, 'price': 2.0450246334075928, 'weight': 0.09221119052696054, 'count_day': 1}, 'SH603296': {'amount': 15484.893560706101, 'price': 1.0897717475891113, 'weight': 0.08441269988331684, 'count_day': 1}, 'SZ000063': {'amount': 11114.201526910563, 'price': 1.5893179178237915, 'weight': 0.08835946323625614, 'count_day': 1}, 'SH

In [30]:
import wk_db.engine


for dt, pos in positions.items():
    print(dt)
    pos = pos.__dict__['position']
    print(f"\tcash: {pos['cash']}")
    print(f"\ttotal value: {pos['now_account_value']}")
    for k, v in pos.items():
        if k in ['cash', 'now_account_value']:
            continue
        s = k
        s = s[2:]+'.'+s[:2]
        # df0 = wk_db.read_sql(f"SELECT * FROM a_share_market where windcode=='{s}' and trade_dt=='{dt}';", db_loc='low_freq_db')
        # # wk_db.
        # df0 = pd.DataFrame(df0.iloc[-1,:]).T
        # close = df0['close_price'].values[0]
        close = v['price']
        # sec_name = df0['sec_name'].values[0]
        # industry_name = df0['industry_name'].values[0]
        # amount = df0['amount'].values[0]
        amount = v['amount']
        # adj_factor = df0['adj_factor'].values[0]
        # instruments = ['SH601398'] 
        fields = ['$factor']
        df = D.features([k], fields, start_time=dt, end_time=dt, freq='day')
        df.columns = ['factor']
        adj_factor = df.values[0][0]
        
        # print(f"\t{k}\t{sec_name}\t{industry_name}\t{close/adj_factor}\t{amount*adj_factor}\t{close*amount}")
        print(f"\t{k}\t{close/adj_factor:.2f}\t{round(amount*adj_factor)}\t{close*amount:.2f}")
        # print(df0)
    # break

    print(f"")

2024-10-15 00:00:00
	cash: 200000
	total value: 200000.0

2024-10-16 00:00:00
	cash: 21253.673401631335
	total value: 199910.67150095035
	SZ000552	2.82	6700	18894.00
	SH601138	23.53	800	18824.00
	SZ002993	27.23	600	16338.00
	SH601872	7.09	2600	18434.00
	SH603296	56.25	300	16875.00
	SZ000063	29.44	600	17664.00
	SH603915	9.41	2000	18820.00
	SH600971	10.02	1800	18036.00
	SH600993	24.60	700	17220.00
	SH600750	21.94	800	17552.00

2024-10-17 00:00:00
	cash: 2170.5553900659725
	total value: 197830.5508125424
	SZ000552	2.77	6700	18559.00
	SH601138	23.71	800	18968.00
	SZ002993	27.22	600	16332.00
	SH601872	6.93	2600	18018.00
	SZ000063	29.45	600	17670.00
	SH603915	9.30	2000	18600.00
	SH600971	9.89	1800	17802.00
	SH600993	23.85	700	16695.00
	SH600750	21.47	800	17176.00
	SH603195	71.68	500	35840.00

2024-10-18 00:00:00
	cash: 2170.5553900659725
	total value: 204648.5537673252
	SZ000552	2.82	6700	18894.00
	SH601138	26.08	800	20864.00
	SZ002993	28.49	600	17094.00
	SH601872	7.05	2600	18330.00
	SZ00006